# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRa
* Model: AutoModelForSequenceClassification
* Evaluation approach: Accuracy
* Fine-tuning dataset: dair-ai/emotion from hugging face

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# !pip3 install -U datasets

The data fields are:

text: a string feature.
label: a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).

In [2]:
from datasets import load_dataset

dataset_name = "dair-ai/emotion"

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset(dataset_name, split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42)

# Show the dataset
ds

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 27720.12 examples/s]


{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 16000
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 2000
 })}

In [3]:
# Inspect the first example. Do you think this is spam or not?
ds["train"][1]

{'text': 'i had pocket qq and was feeling pretty confident lol', 'label': 1}

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_fucntion(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_data = {}
for split in splits:
    tokenized_data[split] = ds[split].map(preprocess_fucntion, batched=True)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 173kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 2.61MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 5.78MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 8.50MB/s]
Map: 100%|██████████| 2000/2000 [00:00<00:00, 2484.47 examples/s]


In [5]:
print(tokenized_data["train"][0]["text"])
print(tokenized_data["train"][0]["label"])
print(tokenized_data["train"][0]["input_ids"])
print(tokenized_data["train"][0]["attention_mask"])

while cycling in the country
4
[101, 2096, 9670, 1999, 1996, 2406, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
# Replace <MASK> with the code to unfreeze all the model parameters

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    # sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},
)

model.safetensors: 100%|██████████| 268M/268M [00:01<00:00, 203MB/s]  
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Set the model's pad token id to match the tokenizer's pad token id
model.config.pad_token_id = tokenizer.pad_token_id

In [8]:
# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.base_model.parameters():
    param.requires_grad = False

In [9]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
model.classifier

Linear(in_features=768, out_features=6, bias=True)

In [11]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/emotion_analysis",
        # Set the learning rate      
        learning_rate=2e-5, # Learning rate for the optimizer.
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=32, # Batch size for training per device.
        per_device_eval_batch_size=32, # Batch size for evaluation per device.
        num_train_epochs=1, # Number of training epochs.
        # weight_decay=0.01, # Weight decay for regularization.
        evaluation_strategy="epoch", # Evaluation is performed at the end of each epoch.
        save_strategy="epoch", # Model is saved at the end of each epoch.
        load_best_model_at_end=True, # Load the best model at the end of training.
    ),
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.571400,1.512514,0.457500


TrainOutput(global_step=500, training_loss=1.571427001953125, metrics={'train_runtime': 326.8909, 'train_samples_per_second': 48.946, 'train_steps_per_second': 1.53, 'total_flos': 2119629570048000.0, 'train_loss': 1.571427001953125, 'epoch': 1.0})

In [12]:
# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()

{'eval_loss': 1.512514352798462,
 'eval_accuracy': 0.4575,
 'eval_runtime': 34.9681,
 'eval_samples_per_second': 57.195,
 'eval_steps_per_second': 1.802,
 'epoch': 1.0}

In [13]:
import pandas as pd
# Convert the tokenized test dataset to a pandas DataFrame
df = pd.DataFrame(tokenized_data["test"])
# Select only the 'sentence' and 'label' columns for simplicity
df = df[["text", "label"]]
# Use the trained model to make predictions on the validation dataset
predictions = trainer.predict(tokenized_data["test"])
# Convert the raw prediction logits to discrete labels (0 or 1 in our case)
# The argmax function is used to select the index (label) with the highest prediction score.
df["predicted_label"] = np.argmax(predictions[0], axis=1)
# Display the first two rows of the dataframe to check the data.
# This shows the actual and predicted labels alongside the sentences.
df.head(10)

,text,label,predicted_label
0,i was feeling really troubled and down over wh...,0,0
1,i feel so thrilled to have three such distingu...,1,1
2,i feel is that the most likeable characters ar...,1,1
3,i tune out the rest of the world and focus on ...,1,1
4,i sit here writing this i feel unhappy inside,0,0
5,im feeling and if ive liked being pregnant,2,1
6,im very hurt and i feel unimportant,0,0
7,i used to be able to hang around talk with the...,3,1
8,i don t have the feeling of divine vibrations,1,1
9,i vented my feelings towards the pathetic excu...,0,1


In [14]:
# Set a pandas display option to show the full content of each column in the DataFrame.
pd.set_option("display.max_colwidth", None)
# Filter the DataFrame to only include rows where the model's predictions do not match the actual labels.
df[df["label"] != df["predicted_label"]].head(10)

,text,label,predicted_label
5,im feeling and if ive liked being pregnant,2,1
7,i used to be able to hang around talk with the cashier when i was putting away my money now i feel rushed and stressed if i take a second to fumble with the coins and put them in my purse,3,1
9,i vented my feelings towards the pathetic excuse of a communicat,0,1
11,i get the feeling that this could be dangerous,3,1
13,i cant write a review for a book i adore unless i am feeling in the adoring mood at that moment,2,1
14,i mentioned in my last blog that i have started to get the feeling that i have been pressured into studying things i do not like which has also made me into a person i might not fully be,4,1
16,i was wondering if you will focus on the problems because any way you are not care for themselves when complaining or feeling needy,0,1
17,im feeling insecure at the moment,4,0
19,i feel like that because for the most part i have accepted that this is a part of my life and that people will never changed,2,1
20,i feel i can only hope im not alone in these thoughts and im sure to all you fellow exchange students you probably have the same thoughts in mind with at least some of this listed some might say being an exchange student is unlike any other experience,0,1


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [15]:
from peft import LoraConfig, get_peft_model, TaskType

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    # sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},
)

# Set the model's pad token id to match the tokenizer's pad token id
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Create a PEFT Config for LoRA
lora_config = LoraConfig(
                        r=8, # Rank
                        lora_alpha=32,
                        target_modules=['q_lin', 'k_lin', 'v_lin'],
                        lora_dropout=0.1,
                        bias="none",
                        task_type=TaskType.SEQ_CLS
                        )

In [18]:
# Convert the model into a PEFT model
lora_model = get_peft_model(model, lora_config)

In [19]:
# print the number of trainable parameters in LoRA
lora_model.print_trainable_parameters()

trainable params: 1,411,596 || all params: 67,774,476 || trainable%: 2.082784085265373


In [21]:
# set format for the training
tokenized_data["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_data["test"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [22]:
tokenized_data['test'][0]

{'label': tensor(0),
 'input_ids': tensor([  101,  1045,  2001,  3110,  2428, 11587,  1998,  2091,  2058,  2054,
          2026,  3611,  2056,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,   

In [23]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/finetuned_emotion_analysis",
        # Set the learning rate      
        learning_rate=2e-5, # Learning rate for the optimizer.
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=32, # Batch size for training per device.
        per_device_eval_batch_size=32, # Batch size for evaluation per device.
        num_train_epochs=3, # Number of training epochs.
        # weight_decay=0.01, # Weight decay for regularization.
        evaluation_strategy="epoch", # Evaluation is performed at the end of each epoch.
        save_strategy="epoch", # Model is saved at the end of each epoch.
        load_best_model_at_end=True, # Load the best model at the end of training.
        remove_unused_columns=False
    ),
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.355000,0.979090,0.652000
2,0.910000,0.766968,0.725500
3,0.791500,0.721907,0.736000


TrainOutput(global_step=1500, training_loss=1.0188471883138022, metrics={'train_runtime': 1904.9236, 'train_samples_per_second': 25.198, 'train_steps_per_second': 0.787, 'total_flos': 6479270313984000.0, 'train_loss': 1.0188471883138022, 'epoch': 3.0})

In [24]:
# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()

{'eval_loss': 0.7219074964523315,
 'eval_accuracy': 0.736,
 'eval_runtime': 35.8408,
 'eval_samples_per_second': 55.802,
 'eval_steps_per_second': 1.758,
 'epoch': 3.0}

In [25]:
# Saving a Trained PEFT Model
lora_model.save_pretrained("lora_model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [26]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("lora_model",  num_labels=6)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
inputs = tokenizer('i had pocket qq and was feeling pretty confident lol', padding="max_length", truncation=True, return_tensors='pt')


In [39]:
import torch

# Make a prediction

# Get predictions (logits)
with torch.no_grad():
    outputs = lora_model(**inputs)
    logits = outputs.logits

In [42]:
# Convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted class (highest probability)
predicted_class = torch.argmax(probabilities, dim=-1).numpy()[0]

print("Predicted class:", predicted_class)


Predicted class: 1
